# Transformer语言模型

本节训练一个 sequence-to-sequence 模型，使用pytorch的 
`nn.Transformer <https://pytorch.org/docs/master/nn.html?highlight=nn%20transformer#torch.nn.Transformer>` module.

PyTorch 1.2 基于论文 `Attention is All YouNeed <https://arxiv.org/pdf/1706.03762.pdf>` 实现了一个 Transformer 模型， ``nn.Transformer`` 模块依赖于 attention 机制实现表达输入和输出文本的关系。


Transformer模型的结构图：
![](../docs/transformer_architecture.jpg)


# 定义模型

基于 ``nn.TransformerEncoder`` 模型训练语言模型。

语言模型任务是为句子后跟随单词输出一个似然概率，表征这个单词可能出现的概率。

首先做 embedding，再做 positional encoding, 表征单词位置关系。``nn.TransformerEncoder`` 由多层`nn.TransformerEncoderLayer <https://pytorch.org/docs/master/nn.html?highlight=transformerencoderlayer#torch.nn.TransformerEncoderLayer>`组成，对于语言模型任务，每个未来可能出现的单词都需要 mask 并预测其概率，为了得到实际的预测单词，``nn.TransformerEncoder``模型的输出后需要接一个 log-Softmax 函数。

In [1]:
import math
import torch
import torch.nn as nn
import torch.nn.functional as F


class TransformerModel(nn.Module):

    def __init__(self, ntoken, ninp, nhead, nhid, nlayers, dropout=0.5):
        super(TransformerModel, self).__init__()
        from torch.nn import TransformerEncoder, TransformerEncoderLayer
        self.model_type = 'Transformer'
        self.src_mask = None
        self.pos_encoder = PositionalEncoding(ninp, dropout)
        encoder_layers = TransformerEncoderLayer(ninp, nhead, nhid, dropout)
        self.transformer_encoder = TransformerEncoder(encoder_layers, nlayers)
        self.encoder = nn.Embedding(ntoken, ninp)
        self.ninp = ninp
        self.decoder = nn.Linear(ninp, ntoken)

        self.init_weights()

    def _generate_square_subsequent_mask(self, sz):
        mask = (torch.triu(torch.ones(sz, sz)) == 1).transpose(0, 1)
        mask = mask.float().masked_fill(mask == 0, float(
            '-inf')).masked_fill(mask == 1, float(0.0))
        return mask

    def init_weights(self):
        initrange = 0.1
        self.encoder.weight.data.uniform_(-initrange, initrange)
        self.decoder.bias.data.zero_()
        self.decoder.weight.data.uniform_(-initrange, initrange)

    def forward(self, src):
        if self.src_mask is None or self.src_mask.size(0) != len(src):
            device = src.device
            mask = self._generate_square_subsequent_mask(len(src)).to(device)
            self.src_mask = mask

        src = self.encoder(src) * math.sqrt(self.ninp)
        src = self.pos_encoder(src)
        output = self.transformer_encoder(src, self.src_mask)
        output = self.decoder(output)
        return output

``PositionalEncoding`` 模块包括 relative 和 absolute 位置编码，positional encodings 与 embeddings 的维度是一样的，这样两者可以相加。

In [2]:
class PositionalEncoding(nn.Module):

    def __init__(self, d_model, dropout=0.1, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)

        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(
            0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:x.size(0), :]
        return self.dropout(x)

# 加载数据

模型训练过程使用来自 torchtext 的Wikitext-2数据集。vocab 基于 train 数据集构建。``batchify（）``函数将数据集排列成列，在将数据划分为大小为`batch_size``的批次后，删除所有剩余的标记。

例如，将字母表作为序列（总长度为26），批量大小为4，我们将字母表分成4个长度为6的序列：

\begin{align}\begin{bmatrix}
  \text{A} & \text{B} & \text{C} & \ldots & \text{X} & \text{Y} & \text{Z}
  \end{bmatrix}
  \Rightarrow
  \begin{bmatrix}
  \begin{bmatrix}\text{A} \\ \text{B} \\ \text{C} \\ \text{D} \\ \text{E} \\ \text{F}\end{bmatrix} &
  \begin{bmatrix}\text{G} \\ \text{H} \\ \text{I} \\ \text{J} \\ \text{K} \\ \text{L}\end{bmatrix} &
  \begin{bmatrix}\text{M} \\ \text{N} \\ \text{O} \\ \text{P} \\ \text{Q} \\ \text{R}\end{bmatrix} &
  \begin{bmatrix}\text{S} \\ \text{T} \\ \text{U} \\ \text{V} \\ \text{W} \\ \text{X}\end{bmatrix}
  \end{bmatrix}\end{align}

In [3]:
import os
import torchtext
from torchtext.data.utils import get_tokenizer

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

TEXT = torchtext.legacy.data.Field(init_token='<sos>',
                                   eos_token='<eos>',
                                   lower=True)

train_txt, val_txt, test_txt = torchtext.legacy.datasets.language_modeling.WikiText2.splits(TEXT)
TEXT.build_vocab(train_txt)

TEXT

In [4]:
len(train_txt.examples[0].text)

2088628

In [5]:
def batchify(data, bsz):
    data = TEXT.numericalize([data.examples[0].text])
    # Divide the dataset into bsz parts.
    nbatch = data.size(0) // bsz
    # Trim off any extra elements that wouldn't cleanly fit (remainders).
    data = data.narrow(0, 0, nbatch * bsz)
    # Evenly divide the data across the bsz batches.
    data = data.view(bsz, -1).t().contiguous()
    return data.to(device)


batch_size = 20
eval_batch_size = 10
train_data = batchify(train_txt, batch_size)
val_data = batchify(val_txt, eval_batch_size)
test_data = batchify(test_txt, eval_batch_size)

print(train_data.shape)
print(val_data.shape)

torch.Size([104431, 20])
torch.Size([21764, 10])


## 定义生成target文本


In [6]:
bptt = 35


def get_batch(source, i):
    seq_len = min(bptt, len(source) - 1 - i)
    data = source[i:i+seq_len]
    target = source[i+1:i+1+seq_len].view(-1)
    return data, target

## 试一下模型效果

设置超参：

In [7]:
ntokens = len(TEXT.vocab.stoi)  # the size of vocabulary
emsize = 200  # embedding dimension
nhid = 200  # the dimension of the feedforward network model in nn.TransformerEncoder
nlayers = 2  # the number of nn.TransformerEncoderLayer in nn.TransformerEncoder
nhead = 2  # the number of heads in the multiheadattention models
dropout = 0.2  # the dropout value
model = TransformerModel(ntokens, emsize, nhead, nhid,
                         nlayers, dropout).to(device)

## 运行模型

In [8]:
import time
criterion = nn.CrossEntropyLoss()
lr = 5.0  # learning rate
optimizer = torch.optim.SGD(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.95)


def train():
    model.train()  # Turn on the train mode
    total_loss = 0.
    start_time = time.time()
    ntokens = len(TEXT.vocab.stoi)
    for batch, i in enumerate(range(0, train_data.size(0) - 1, bptt)):
        data, targets = get_batch(train_data, i)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output.view(-1, ntokens), targets)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.5)
        optimizer.step()

        total_loss += loss.item()
        log_interval = 200
        if batch % log_interval == 0 and batch > 0:
            cur_loss = total_loss / log_interval
            elapsed = time.time() - start_time
            print('| epoch {:3d} | {:5d}/{:5d} batches | '
                  'lr {:02.2f} | ms/batch {:5.2f} | '
                  'loss {:5.2f} | ppl {:8.2f}'.format(
                      epoch, batch, len(
                          train_data) // bptt, scheduler.get_lr()[0],
                      elapsed * 1000 / log_interval,
                      cur_loss, math.exp(cur_loss)))
            total_loss = 0
            start_time = time.time()


def evaluate(eval_model, data_source):
    eval_model.eval()  # Turn on the evaluation mode
    total_loss = 0.
    ntokens = len(TEXT.vocab.stoi)
    with torch.no_grad():
        for i in range(0, data_source.size(0) - 1, bptt):
            data, targets = get_batch(data_source, i)
            output = eval_model(data)
            output_flat = output.view(-1, ntokens)
            total_loss += len(data) * criterion(output_flat, targets).item()
    return total_loss / (len(data_source) - 1)

在validation loss最优时保存模型，在每个epoch结束时调整learning rate。

In [ ]:
best_val_loss = float("inf")
epochs = 10  # The number of epochs
best_model = None
MODEL_PATH = 'transformer_lm.pth'
for epoch in range(1, epochs + 1):
    epoch_start_time = time.time()
    train()
    val_loss = evaluate(model, val_data)
    print('-' * 89)
    print('| end of epoch {:3d} | time: {:5.2f}s | valid loss {:5.2f} | '
          'valid ppl {:8.2f}'.format(epoch, (time.time() - epoch_start_time),
                                     val_loss, math.exp(val_loss)))
    print('-' * 89)

    if val_loss < best_val_loss:
        best_val_loss = val_loss
        best_model = model
        torch.save(best_model.state_dict(), MODEL_PATH)

    scheduler.step()

best_model.load_state_dict(torch.load(MODEL_PATH))

/Users/xuming/opt/anaconda3/lib/python3.8/site-packages/torch/optim/lr_scheduler.py:369: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "


| epoch   1 |   200/ 2983 batches | lr 5.00 | ms/batch 491.00 | loss  8.02 | ppl  3046.57
| epoch   1 |   400/ 2983 batches | lr 5.00 | ms/batch 499.58 | loss  6.85 | ppl   941.26
| epoch   1 |   600/ 2983 batches | lr 5.00 | ms/batch 500.60 | loss  6.42 | ppl   614.55
| epoch   1 |   800/ 2983 batches | lr 5.00 | ms/batch 496.18 | loss  6.29 | ppl   537.29
| epoch   1 |  1000/ 2983 batches | lr 5.00 | ms/batch 487.90 | loss  6.18 | ppl   481.80
| epoch   1 |  1200/ 2983 batches | lr 5.00 | ms/batch 496.27 | loss  6.15 | ppl   467.33
| epoch   1 |  1400/ 2983 batches | lr 5.00 | ms/batch 506.12 | loss  6.09 | ppl   441.26
| epoch   1 |  1600/ 2983 batches | lr 5.00 | ms/batch 514.11 | loss  6.10 | ppl   447.08
| epoch   1 |  1800/ 2983 batches | lr 5.00 | ms/batch 510.11 | loss  5.99 | ppl   400.76


Evaluate the model with the test dataset
-------------------------------------

Apply the best model to check the result with the test dataset.



In [ ]:
test_loss = evaluate(best_model, test_data)
print('=' * 89)
print('| End of training | test loss {:5.2f} | test ppl {:8.2f}'.format(
    test_loss, math.exp(test_loss)))
print('=' * 89)

In [ ]:
import os
os.remove('transformer_lm.pth')

本节完。